# Dataset Builder v1 — Boletín Oficial CABA (Train / Test / Val)

Genera candidatos (fragmentos) **para etiquetar** a partir de PDFs del Boletín Oficial de CABA.

**Características**
- Lee todos los PDF de la carpeta de origen en Drive en **orden lexicográfico**.
- **Ventana por oraciones**: toma la oración donde aparece el *hit* y **2 oraciones** a cada lado (máx. ~400–600 palabras).
- Busca candidatos por **VERBO**, **KEYWORD**, **NORMAR** (configurable). **TEMAS** se usa **solo** para anotar la columna `temas`.
- Limpieza mínima (espacios, deduplicación por hash).
- **CSV único** (append idempotente) con columnas fijas; `label` queda **vacío** para etiquetar manualmente.
- **Mueve** el PDF procesado a la carpeta correspondiente.
- **Barra de progreso** (tqdm) y resumen por archivo.


In [ ]:
# ===== 1) Dependencias e importaciones =====
!pip -q install pypdf nltk tqdm > /dev/null
from google.colab import drive
drive.mount('/content/drive')

import os, re, csv, hashlib, shutil
from typing import List, Tuple, Set

import pandas as pd
from pypdf import PdfReader
from tqdm import tqdm
import re
import nltk
nltk.download('punkt', quiet=True)
try:
    nltk.download('punkt_tab', quiet=True)
except Exception as _e:
    print('Aviso: no se pudo descargar punkt_tab automáticamente:', _e)
from nltk.tokenize import sent_tokenize

print('Entorno listo ✅')

Mounted at /content/drive
Entorno listo ✅


In [ ]:
# ===== 2) Configuración =====
# >>> EDITABLE <<<

# Carpetas en Google Drive
RAW_DIR = "/content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/data/raw/test_2025"
PROCESSED_DIR = "/content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/data/processed/boletines_procesados/test"
CSV_PATH = "/content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/data/labels/etiquetas_v3etiquetas_test_H2_2025.csv"

# Split fijo para este lote
SPLIT_TAG = "test"

# Ventana por oraciones
WINDOW_SENTENCES_LEFT = 2
WINDOW_SENTENCES_RIGHT = 2
MAX_WORDS_WINDOW = 600  # recortará si supera este máximo
MIN_WORDS_WINDOW = 80   # si queda muy corta, intentará expandir si es posible

# Listas de patrones (completalas vos)
VERBOS_ACCION = [
    r"\b[Mm]odifica\b", r"\b[Mm]odifícase\b", r"\b[Mm]odificar\b",
    r"\b[Dd]eroga\b", r"\b[Dd]erogar\b", r"\b[Dd]erógase\b",
    r"\b[Aa]prueba\b", r"\b[Aa]probar\b", r"\b[Aa]pruébese\b",
    r"\b[Dd]eja sin efecto\b", r"\b[Dd]ejar sin efecto\b", r"\b[Dd]éjase sin efecto\b",
    r"\b[Ss]ustituye\b", r"\b[Ss]ustituir\b", r"\b[Ss]ustitúyese\b",
    r"\b[Ee]stablece\b", r"\b[Ee]stablecer\b", r"\b[Ee]stablécese\b",
    r"\b[Ff]ija\b", r"\b[Ff]ijar\b", r"\b[Ff]íjese\b",
    r"\b[Dd]etermina\b", r"\b[Dd]etermínase\b", r"\b[Dd]eterminar\b",
    r"\b[Rr]eglamenta\b", r"\b[Rr]eglaméntese\b", r"\b[Rr]eglamentación\b",
    r"\b[Pp]rorroga\b", r"\b[Pp]rorrógase\b", r"\b[Pp]rorrogar\b"
]

KEYWORDS = [
    # Construcción
    "código de edificación",
    "reglamentos técnicos",
    "reglamento técnico",
    "planos de mensura",
    "planos de obra",
    "planos de instalaciones",
    "obras en contravención",
    "accesibilidad",
    "conservación de fachadas",

    # Digestos normativos
    "compendio normativo",
    "digesto",

    # Escuelas seguras
    "ueresgp",
    "régimen de escuelas seguras de gestión privada",

    # Espacio público
    "uso del espacio público",
    "publicidad exterior",

    # Fiscal y tarifario
    "ley tarifaria",
    "código fiscal",
    "unidad tarifaria",
    "derecho para el desarrollo urbano y el hábitat sustentable",
    "derecho de construcción sustentable",
    "derecho de construcción",

    # Habilitaciones
    "código de habilitaciones",
    "autorización de actividad económica",
    "autorización de actividades económicas",
    "autorizaciones de actividades económicas",
    "habilitación",
    "habilitación económica",
    "ley marco de regulación de actividades económicas",

    # Impacto ambiental
    "impacto ambiental",
    "certificado de aptitud ambiental",

    # Sistema de Autoprotección
    "sistema de autoprotección",
    "sistemas de autoprotección",

    # Urbanismo
    "código urbanístico",
    "catastro",
    "área céntrica",
    "reurbanización",
    "zonificación",
    "planeamiento urbano",
    "normas urbanísticas",
    "convenios urbanísticos",
    "convenio urbanístico"
]

PATRONES_NORMAR = [
    r"[Oo]rdenanza(?: [Nn]°?)? ?3\.?442",
    r"[Oo]rdenanza(?: [Nn]°?)? ?3\.?421",
    r"[Dd]isposici[oó]n(?: [Nn]°?)? ?3\.?500(?:[-/]?GCABA)?[-/]?DGOEP[-/]?16",
    r"[Dd]isposici[oó]n(?: [Nn]°?)? ?331(?:[-/]?GCABA)?[-/]?DGDCIV[-/]?25",
    r"[Dd]isposici[oó]n(?: [Nn]°?)? ?89(?:[-/]?GCABA)?[-/]?DGROC[-/]?24",
    r"[Dd]isposici[oó]n(?: [Nn]°?)? ?526(?:[-/]?GCABA)?[-/]?DGFYCO[-/]?24",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?275(?:[-/]?GCABA)?[-/]?APRA[-/]?23",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?188(?:[-/]?GCABA)?[-/]?SSGU[-/]?24",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?160(?:[-/]?GCABA)?[-/]?SSHA[-/]?24",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?96(?:[-/]?GCABA)?[-/]?AGC[-/]?25",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?345(?:[-/]?GCABA)?[-/]?AGC[-/]?21",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?103(?:[-/]?GCABA)?[-/]?APRA[-/]?25",
    r"[Rr]esoluci[oó]n(?: [Nn]°?)? ?1(?:[-/]?GCABA)?[-/]?MEPHUGC[-/]?25",
    r"[Dd]ecreto(?: [Nn]°?)? ?51/18",
    r"[Dd]ecreto(?: [Nn]°?)? ?86/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?87/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?99/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?105/19",
    r"[Dd]ecreto(?: [Nn]°?)? ?475/20",
    r"[Dd]ecreto(?: [Nn]°?)? ?129/25",
    r"[Dd]ecreto(?: [Nn]°?)? ?116/25",
    r"[Dd]ecreto(?: [Nn]°?)? ?164/25",
    r"[Dd]ecreto(?: [Nn]°?)? ?189/25",
    r"[Ll]ey(?: [Nn]°?)? ?123",
    r"[Ll]ey(?: [Nn]°?)? ?2\.?189",
    r"[Ll]ey(?: [Nn]°?)? ?2\.?936",
    r"[Ll]ey(?: [Nn]°?)? ?5\.?920",
    r"[Ll]ey(?: [Nn]°?)? ?6\.101",
    r"[Ll]ey(?: [Nn]°?)? ?6\.437",
    r"[Ll]ey(?: [Nn]°?)? ?6\.776",
    r"[Ll]ey(?: [Nn]°?)? ?6\.779",
    r"[Ll]ey(?: [Nn]°?)? ?6\.099",
    r"[Ll]ey(?: [Nn]°?)? ?6\.100",
    r"[Ll]ey(?: [Nn]°?)? ?6\.438",
    r"[Ll]ey(?: [Nn]°?)? ?6\.508",
    r"[Ll]ey(?: [Nn]°?)? ?6\.509",
    r"[Ll]ey(?: [Nn]°?)? ?6\.806",
    r"[Ll]ey(?: [Nn]°?)? ?6\.769"
]

TEMAS = [
  "construcción",
  "digestos normativos",
  "escuelas seguras",
  "espacio público",
  "fiscal y tarifario",
  "habilitaciones",
  "impacto ambiental",
  "sistema de autoprotección",
  "urbanismo"
]

# Columnas del CSV (orden fijo)
CSV_COLUMNS = [
    "id","split","fecha","anio","boletin_nro","origen_pdf","fuente_url","organismo_emisor","tipo_figura_normativa",
    "titulo","fragmento","label","anotador","notas","is_ambiguo","deroga_modifica_ref","numero_norma","temas","origen_flags"
]

# Crear carpetas si no existen
os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Aviso útil
if len(VERBOS_ACCION)==0 and len(KEYWORDS)==0 and len(PATRONES_NORMAR)==0:
    print('⚠️ Advertencia: VERBOS_ACCION, KEYWORDS y PATRONES_NORMAR están vacías. No se generarán fragmentos hasta completarlas.')

print('Configuración cargada ✅')

Configuración cargada ✅


In [ ]:
# ===== 3) Utilidades =====
def read_pdf_text_by_page(pdf_path: str) -> List[str]:
    texts = []
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            t = page.extract_text() or ""
            texts.append(t)
    except Exception as e:
        print(f"[ERROR] Leyendo PDF: {pdf_path}: {e}")
        return []
    return texts

def sent_segment(text: str) -> List[str]:
    """
    Segmenta en oraciones (NLTK español) con fallback regex.
    Intenta con 'spanish' (punkt_tab); si falta, descarga y reintenta.
    Si falla igual, usa un separador regex simple.
    """
    try:
        sents = sent_tokenize(text, language='spanish')
    except LookupError:
        # Reintenta descargando recursos
        try:
            nltk.download('punkt', quiet=True)
            nltk.download('punkt_tab', quiet=True)
            sents = sent_tokenize(text, language='spanish')
        except Exception:
            # Fallback regex (sin imports dentro de la función)
            sents = re.split(r'(?<=[.!?])\s+(?=[A-ZÁÉÍÓÚÑ])', text)
    except Exception:
        sents = re.split(r'(?<=[.!?])\s+(?=[A-ZÁÉÍÓÚÑ])', text)

    sents = [re.sub(r"\s+", " ", s).strip() for s in sents if s and s.strip()]
    return sents

def normalize_for_hash(s: str) -> str:
    s = s.lower()
    s = re.sub(r"\s+", " ", s).strip()
    return s

def sha1(s: str) -> str:
    return hashlib.sha1(s.encode('utf-8')).hexdigest()

def any_match(patterns: List[str], text: str) -> bool:
    for pat in patterns:
        if re.search(pat, text, flags=re.IGNORECASE):
            return True
    return False

def window_by_sentences(sents: List[str], idx: int, k_left=2, k_right=2, max_words=600) -> str:
    left = max(0, idx - k_left)
    right = min(len(sents), idx + k_right + 1)
    window = sents[left:right]
    text = " ".join(window)
    words = text.split()
    if len(words) > max_words:
        while len(words) > max_words and (k_left > 0 or k_right > 0) and (right - left > 1):
            if k_right >= k_left:
                right -= 1
            else:
                left += 1
            window = sents[left:right]
            text = " ".join(window)
            words = text.split()
    return text

def try_extract_fecha_from_filename(name: str) -> Tuple[str, str]:
    name = name.replace(" ", "_")
    m = re.search(r"(20\d{2})[-_](\d{2})[-_](\d{2})", name)
    if m:
        y, mo, d = m.group(1), m.group(2), m.group(3)
        return f"{y}-{mo}-{d}", y
    m = re.search(r"(\d{2})[-_](\d{2})[-_](20\d{2})", name)
    if m:
        d, mo, y = m.group(1), m.group(2), m.group(3)
        return f"{y}-{mo}-{d}", y
    m = re.search(r"(20\d{2})(\d{2})(\d{2})", name)
    if m:
        y, mo, d = m.group(1), m.group(2), m.group(3)
        return f"{y}-{mo}-{d}", y
    m = re.search(r"(20\d{2})", name)
    if m:
        y = m.group(1)
        return "", y
    return "", ""

def extract_numero_norma(text: str) -> str:
    m = re.search(r"\b(ley|decreto|resoluci(ó|o)n|disposici(ó|o)n|ordenanza)\b\s*(n[º°o]?\s*\d+[./-]?\d*)", text, re.IGNORECASE)
    if m:
        return m.group(0)
    m = re.search(r"\bN[º°o]\s*\d+[./-]?\d*\b", text, re.IGNORECASE)
    if m:
        return m.group(0)
    return ""

def extract_temas(text: str) -> str:
    # Usa las expresiones de TEMAS para anotar, no para disparar
    found = []
    for pat in TEMAS:
        hits = re.findall(pat, text, flags=re.IGNORECASE)
        if hits:
            found.append(pat)
    seen = set()
    uniq = []
    for f in found:
        if f not in seen:
            seen.add(f)
            uniq.append(f)
    return ";".join(uniq)

def has_deroga_modifica(text: str) -> str:
    if re.search(r"\b(deroga(n|r|do|da|das|dos)?|modifica(n|r|do|da|das|dos)?|sustituye(n|r|do|da|das|dos)?)\b", text, re.IGNORECASE):
        return "1"
    return "0"

def ensure_csv(csv_path: str, columns: List[str]):
    if not os.path.exists(csv_path):
        with open(csv_path, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(columns)

def load_existing_keys(csv_path: str) -> Set[str]:
    keys = set()
    if os.path.exists(csv_path):
        try:
            for chunk in pd.read_csv(csv_path, chunksize=5000, dtype=str, keep_default_na=False):
                for _, row in chunk.iterrows():
                    frag = (row.get("fragmento", "") or "")
                    fecha = (row.get("fecha", "") or "")
                    origen = (row.get("origen_pdf", "") or "")
                    key = sha1(normalize_for_hash(frag) + "|" + fecha + "|" + origen)
                    keys.add(key)
        except Exception as e:
            print(f"[WARN] No se pudieron cargar todas las llaves existentes: {e}")
    return keys

def append_rows_csv(csv_path: str, rows: List[List[str]]):
    with open(csv_path, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        for r in rows:
            w.writerow(r)

print('Utilidades listas ✅')

Utilidades listas ✅


In [ ]:
# ===== 4) Pipeline principal =====
def process_pdf(pdf_path: str, existing_keys: Set[str]) -> Tuple[int, int]:
    """
    Procesa un PDF y devuelve (filas_nuevas_agregadas, descartadas_locales).
    - Gating sobre la ORACIÓN (sent):
        Aceptar si:
          1) contiene KEYWORDS, o
          2) contiene PATRONES_NORMAR, o
          3) VERBOS_ACCION y PATRONES_NORMAR, o
          4) KEYWORDS y PATRONES_NORMAR
        (Nunca aceptar VERBOS_ACCION solo)
    - Agrupa oraciones disparadoras contiguas en un solo segmento -> una ventana por segmento.
    - Evita ventanas solapadas fuertes (IoU > 0.6) en la misma página.
    - Filtro final por VENTANA: si quedó solo VERBO, descarta.
    """
    base = os.path.basename(pdf_path)
    fecha_iso, anio = try_extract_fecha_from_filename(base)

    pages = read_pdf_text_by_page(pdf_path)
    if not pages:
        # mover el PDF aunque no se pudo leer, para que no se repita
        try:
            dest = os.path.join(PROCESSED_DIR, base)
            if os.path.exists(dest):
                root, ext = os.path.splitext(base)
                dest = os.path.join(PROCESSED_DIR, f"{root}__moved{ext}")
            shutil.move(pdf_path, dest)
        except Exception as e:
            print(f"[WARN] No se pudo mover {base} a processed: {e}")
        return 0, 0

    new_rows = []
    discarded_local = 0

    # dedup dentro del PDF por contenido normalizado
    seen_hashes_local = set()

    for p_idx, page_text in enumerate(pages):
        if not page_text or not page_text.strip():
            continue

        sents = sent_segment(page_text)
        if not sents:
            continue

        # ---------- 1) GATING por ORACIÓN: recolectar índices disparadores ----------
        triggers_idx = []
        for i, sent in enumerate(sents):
            has_verbo   = any_match(VERBOS_ACCION, sent)
            has_kw      = any_match(KEYWORDS, sent)
            has_normar  = any_match(PATRONES_NORMAR, sent)

            has_candidate = (
                has_kw
                or has_normar
                or (has_verbo and has_normar)
                or (has_kw and has_normar)
            )
            # Blindaje extra: nunca verbo solo
            if has_verbo and not (has_kw or has_normar):
                has_candidate = False

            if has_candidate:
                triggers_idx.append(i)

        if not triggers_idx:
            continue

        # ---------- 2) AGRUPAR ÍNDICES CONTIGUOS EN SEGMENTOS ----------
        # Permite una "grieta" de hasta 1 oración entre disparos para agrupar
        GAP = 1
        segments = []
        start = prev = triggers_idx[0]
        for idx in triggers_idx[1:]:
            if idx - prev <= GAP:
                prev = idx
            else:
                segments.append((start, prev))
                start = prev = idx
        segments.append((start, prev))  # último segmento

        # Para evitar ventanas muy solapadas en la misma página
        spans_seen_page: List[Tuple[int,int]] = []

        # Utilidad local: IoU de rangos de oraciones [L,R]
        def iou(span_a: Tuple[int,int], span_b: Tuple[int,int]) -> float:
            aL, aR = span_a
            bL, bR = span_b
            inter = max(0, min(aR, bR) - max(aL, bL) + 1)
            union = (aR - aL + 1) + (bR - bL + 1) - inter
            return inter / union if union else 0.0

        for (seg_start, seg_end) in segments:
            # ---------- 3) Construir ventana por oraciones ----------
            left  = max(0, seg_start - WINDOW_SENTENCES_LEFT)
            right = min(len(sents) - 1, seg_end + WINDOW_SENTENCES_RIGHT)

            # Evitar ventanas fuertemente solapadas con ya aceptadas (IoU > 0.6)
            if any(iou((left, right), span) > 0.6 for span in spans_seen_page):
                discarded_local += 1
                continue
            spans_seen_page.append((left, right))

            # Construir texto de la ventana y recortar por MAX_WORDS_WINDOW si hace falta
            window_sents = sents[left:right+1]
            window = " ".join(window_sents)
            words = window.split()
            if len(words) > MAX_WORDS_WINDOW:
                # recorta simétrico manteniendo oraciones completas
                L, R = left, right
                while len(words) > MAX_WORDS_WINDOW and (R - L) > 0:
                    # recorta del lado más largo respecto al centro del segmento
                    if (R - seg_end) >= (seg_start - L):
                        R -= 1
                    else:
                        L += 1
                    window_sents = sents[L:R+1]
                    window = " ".join(window_sents)
                    words = window.split()
                left, right = L, R  # actualizar por consistencia

            # Si quedó demasiado corta, intenta expandir un paso si es posible
            if len(words) < MIN_WORDS_WINDOW:
                L = max(0, left - 1)
                R = min(len(sents) - 1, right + 1)
                window = " ".join(sents[L:R+1])
                left, right = L, R

            # ---------- 4) Flags por VENTANA y filtro "solo VERBO" ----------
            flags = []
            if any_match(VERBOS_ACCION, window):     flags.append("VERBO")
            if any_match(KEYWORDS, window):          flags.append("KEYWORD")
            if any_match(PATRONES_NORMAR, window):   flags.append("NORMAR")

            # Si en la ventana quedó solo VERBO, descartar
            if ("VERBO" in flags) and ("KEYWORD" not in flags) and ("NORMAR" not in flags):
                discarded_local += 1
                continue

            # ---------- 5) Deduplicaciones ----------
            window_norm = normalize_for_hash(window)
            h = sha1(window_norm)
            if h in seen_hashes_local:
                discarded_local += 1
                continue
            seen_hashes_local.add(h)

            uniq_key = sha1(window_norm + "|" + (fecha_iso or "") + "|" + base)
            if uniq_key in existing_keys:
                discarded_local += 1
                continue

            # ---------- 6) Extraer metadatos y armar fila ----------
            numero_norma = extract_numero_norma(window)
            temas_detect = extract_temas(window)  # TEMAS solo anota, no dispara
            deroga_modifica_ref = has_deroga_modifica(window)
            origen_flags = ";".join(flags)

            # ID estable: fecha + página + span de oraciones + hash corto
            id_str = f"{(fecha_iso or 'NA').replace('-', '')}-p{p_idx:03d}-s{left:03d}-e{right:03d}-h{h[:8]}"

            row = [
                id_str,          # id
                "train",         # split (ajusta si estás corriendo val/test)
                fecha_iso,       # fecha
                anio,            # anio
                "",              # boletin_nro
                base,            # origen_pdf
                "",              # fuente_url
                "",              # organismo_emisor
                "",              # tipo_figura_normativa
                "",              # titulo
                window,          # fragmento
                "",              # label (vacío)
                "",              # anotador
                "",              # notas
                "0",             # is_ambiguo
                deroga_modifica_ref,  # deroga_modifica_ref
                numero_norma,    # numero_norma
                temas_detect,    # temas
                origen_flags,    # origen_flags
            ]
            new_rows.append(row)

    # ---------- 7) Append CSV y mover PDF ----------
    appended = 0
    if new_rows:
        append_rows_csv(CSV_PATH, new_rows)
        # actualizar llaves globales para evitar reinsertar si re-corremos
        for r in new_rows:
            frag  = r[CSV_COLUMNS.index("fragmento")]
            fecha = r[CSV_COLUMNS.index("fecha")] or ""
            origen = r[CSV_COLUMNS.index("origen_pdf")] or ""
            key = sha1(normalize_for_hash(frag) + "|" + fecha + "|" + origen)
            existing_keys.add(key)
        appended = len(new_rows)

    try:
        dest = os.path.join(PROCESSED_DIR, base)
        if os.path.exists(dest):
            root, ext = os.path.splitext(base)
            dest = os.path.join(PROCESSED_DIR, f"{root}__moved{ext}")
        shutil.move(pdf_path, dest)
    except Exception as e:
        print(f"[WARN] No se pudo mover {base} a processed: {e}")

    return appended, discarded_local


def run_batch(limit_pdfs=None):
    ensure_csv(CSV_PATH, CSV_COLUMNS)
    existing = load_existing_keys(CSV_PATH)

    pdfs = [os.path.join(RAW_DIR, f) for f in os.listdir(RAW_DIR) if f.lower().endswith('.pdf')]
    pdfs.sort()

    total_new = 0
    total_discard = 0

    if not pdfs:
        print("No hay PDFs para procesar en:", RAW_DIR)
        return

    for idx, pdf in enumerate(tqdm(pdfs, desc="Procesando PDFs", unit="pdf")):
        if limit_pdfs is not None and idx >= limit_pdfs:
            tqdm.write(f"Corte manual: solo {limit_pdfs} PDFs.")
            break
        base = os.path.basename(pdf)
        new_rows, disc = process_pdf(pdf, existing)
        total_new += new_rows
        total_discard += disc
        tqdm.write(f"Analizado: {base} | filas nuevas: {new_rows}")

    print("\nResumen")
    print("PDFs procesados:", min(len(pdfs), limit_pdfs) if limit_pdfs else len(pdfs))
    print("Filas nuevas agregadas:", total_new)
    print("Descartadas por duplicado (local):", total_discard)
    print("CSV:", CSV_PATH)
    print("Processed dir:", PROCESSED_DIR)

print('Pipeline listo ✅')

Pipeline listo ✅


In [ ]:
# ===== 5) Ejecutar =====
run_batch(limit_pdfs=8)

No hay PDFs para procesar en: /content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/data/raw/test_2025
